In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.3.0


## Load the data

In [10]:
training_dir = 'chest_xray/chest_xray/train'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  training_dir,
  seed=123,
  image_size=(200,200),
  batch_size=4)

validation_dir = 'chest_xray/chest_xray/val'
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  validation_dir,
  seed=123,
  image_size=(200,200),
  batch_size=4)

testing_dir = 'chest_xray/chest_xray/test'
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  testing_dir,
  seed=123,
  image_size=(200,200),
  batch_size=4)

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [11]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(4, 200, 200, 3)
(4,)


## Standardize the data

In [12]:
# Instatiate the normalization layer
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

## Configure dataset for performance

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_tensors = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_tensors = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Train a model

In [15]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [18]:
model.compile(
  optimizer='adam',
#   loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss=tf.losses.BinaryCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [19]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
1304/1304 [==============================] - 169s 129ms/step - loss: 7.6889 - accuracy: 0.7429 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 2/3
 513/1304 [==========>...................] - ETA: 1:49 - loss: 7.6903 - accuracy: 0.7481

KeyboardInterrupt: 